## Import Dependancies

In [22]:
#scraping tools
from urllib.error import HTTPError
from datetime import datetime
import json
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import numpy as np
import requests

# NLP tools
import pandas as pd
from collections import Counter
import re
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

#import API key
from keys import magic_key

#viz tools
import matplotlib.pyplot as plt
import seaborn as sns

### Get YouTube Transcript (Currently Broken)

Apparently you can't scrape the transcripts the old fashioned way. 

In [11]:
# print("Drop in the link to the youtube video.")
# url = input()

# # https://www.youtube.com/watch?v=jCZKEnThrjI

In [12]:
# print (f'Your url is {url}')
# with request.urlopen(url) as response:
#     resp = response.read()
#     soup = bs(resp, "html.parser")
#     print("Your youtube soup is ready!")

In [13]:
# print(soup)

In [14]:
# #pull in title and transcript from the soup
# lil_soup = soup.find_all("div", id_="segments-container")
# print(lil_soup)

### Functions that Returns Youtube Comments
Uses the Youtube API

In [72]:
#function for getting Youtube comments with the Youtube API
def get_comments():
    
    #get the parameters url from the user and create a 
    video_url = input("Drop in the Youtube Link: ")
    print ('Got it! Thanks')
    video_id = video_url[32:] #extract the video id from the url
    # print(video_id)
    
    # init API query
    comment_url = f"https://www.googleapis.com/youtube/v3/commentThreads?key={magic_key}"
    params = f"&textFormat=plainText&part=snippet&videoId={video_id}&maxResults=300"
    title_url = f'https://youtube.googleapis.com/youtube/v3/videos?part=snippet&id={video_id}&key={magic_key}'
    
    # 1ST API CALL
    response = requests.get(title_url).json()
    video_title = response['items'][0]['snippet']['title']
    
    # 2ND API CALL
    query_url = comment_url + params
    response = requests.get(query_url).json()
    
    
    # LOOP THRU COMMENTS AND GET THAT YUMMY COMMENT TEXT
    # response['items'][10] # this is the format of a SINGLE COMM
    #raw_comment_list = response['items']

    num_comments = response['pageInfo']['totalResults'] # number of comments acutally returned from API
    counter = np.arange(0,num_comments) # create list to count thru
    print(f'We found {num_comments} comments for {video_title}.')

    comments = [video_title]
    for c in counter:
        comment_text = response['items'][c]['snippet']['topLevelComment']['snippet']['textOriginal']
        #print(comment_text)
        #print('------------------')
        comments.append(comment_text)
        
    return comments

In [ ]:
get_comments()

In [ ]:
# testing function
comments = get_comments()

### Functions to Determine Sentiment

In [ ]:
def get_sentimental(string_list):
    sia = SentimentIntensityAnalyzer()

    #init lists
    text = []
    neg_score = []
    neu_score = []
    pos_score = []
    compound_score = []

    for string in string_list:
        siaOut = sia.polarity_scores(string)

        text.append(string)
        neg_score.append(siaOut['neg'])
        neu_score.append(siaOut['neu'])
        pos_score.append(siaOut['pos'])
        compound_score.append(siaOut['compound'])

    #     print(comment)
    #     print(siaOut)
    #     print('-------------')

    # create a dataframe of the comments and their sentiments
    df = pd.DataFrame({'text': text,
                       'neg_score': neg_score,
                       'neu_score': neu_score,
                       'pos_score': pos_score,
                       'compound_score': compound_score
                      })
    return df

In [ ]:
test = get_sentimental(comments)

In [ ]:
def main():
    comments = get_comments()
    df = get_sentimental(comments)
    
    return df

## Positive Comment Test

In [ ]:
# test on video with positive comments
# https://www.youtube.com/watch?v=jCZKEnThrjI&t=7s (Caitlin Doughty and Chelsea Fagan Podcast)

test = main()

In [ ]:
print(f'Average Compound Score {test.compound_score.mean()}')

In [ ]:
# scatterplot test (NOT GREAT)
plt.scatter(test.index, test.compound_score)

In [ ]:
# swarmplot test (BETTER)
sns.swarmplot(x=test.compound_score)

In [ ]:
#density plot (WHAT IS THIS EVEN)
test.compound_score.plot(kind='density')

In [ ]:
#violinplot test (I LIKE IT BUT NEED MORE INFO)
sns.violinplot(x=test.compound_score)

In [ ]:
#histogram test
plt.hist(test.compound_score)

## Negative Comment Test

In [ ]:
# test on video with positive comments
# https://www.youtube.com/watch?v=YbJOTdZBX1g&t (2018 Youtube Rewind)

test_2 = main()

In [ ]:
print(f'Average Compound Score {test_2.compound_score.mean()}')

In [ ]:
# scatterplot test (NOT GREAT)
plt.scatter(test_2.index, test_2.compound_score, color='r')

In [ ]:
# swarmplot test (BETTER)
sns.swarmplot(x=test_2.compound_score, color='r')

In [ ]:
#density plot (WHAT IS THIS EVEN)
test_2.compound_score.plot(kind='density',color='r')

In [ ]:
#violinplot test (I LIKE IT BUT NEED MORE INFO)
sns.violinplot(x=test_2.compound_score, color='r')

In [ ]:
#histogram test
plt.hist(test_2.compound_score, color='r')

## Comparing the Two Videos

In [ ]:
test['video'] = 'Money and Death Podcast'

In [ ]:
test_2['video'] = '2018 Youtube Rewind'

In [ ]:
compare_df = pd.concat([test,test_2])

In [ ]:
compare_df['dummy'] = 0

In [ ]:
plot = sns.violinplot(data=compare_df, y='compound_score', split=True, hue='video', x='dummy')
picture = plot.get_figure()
picture.savefig('output.png')

### Tokenize Document

In [ ]:
file = "Resources/Corpus_Podcast"

In [ ]:
with open(file) as f:
    raw_text = f.readlines()

In [ ]:
def to_tokens(dirty_text):
    
    big_string = ' '.join(dirty_text).lower() # join and lower into big string
    big_string = re.sub(r"[^a-z0-9'\s]", '', big_string) #drop punctuation
    big_string = re.sub(r'\d+', '', big_string) # remove numbers
    
    tokens = nltk.word_tokenize(big_string) # tokenize with nltk
    print(f'Your text is now {len(tokens)} tokens.')
    
    print("Drop it like it's stop.")
    #loop thru words and strip stop words
    cleaned_words = []
    for token in tokens:
        if token not in stopwords.words('english'):
            cleaned_words.append(token)
            
    print(f'Now your text is only {len(cleaned_words)} tokens.')
    
    return tokens

In [ ]:
tokens = to_tokens(raw_text)
tokens

In [ ]:
def score_text(dirty_text):
    
    big_string = ' '.join(dirty_text).lower() # join and lower into big string
    big_string = big_string.replace('.','').replace(',','') #drop commas and periods
    
    sia = SentimentIntensityAnalyzer()
    siaOut = sia.polarity_scores(big_string)
    print(siaOut)

In [ ]:
score_text(tokens)

In [ ]:
score_text(raw_text)